In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm, trange
from math import sqrt, inf
import pandas as pd
import numpy as np
import heapq
import random
import math

In [2]:
n,m=943,1682
cols = ['user_id', 'item_id', 'rating', 'ts']
base = pd.read_csv('../../ml-100k/u1.base',sep='\t',names=cols)
test = pd.read_csv('../../ml-100k/u1.test',sep='\t',names=cols)

In [3]:
# 创建uesr-item矩阵
train_matrix = np.zeros((n, m))
test_matrix = np.zeros((n, m))
y_ui = np.zeros((n, m))
for i in base.itertuples():
    if i[3]>=4:
        train_matrix[i[1] - 1, i[2] - 1] = i[3]

for i in test.itertuples():
    if i[3]>=4:
        test_matrix[i[1] - 1, i[2] - 1] = i[3]

# 创建y矩阵
for i in range(n):
    for j in range(m):
        if train_matrix[i,j] >= 4:
            y_ui[i,j] = 1
# 确定物品集与用户集
I = list(np.where(sum(train_matrix+test_matrix)!=0)[0])
U = list(np.where(sum(train_matrix.T+test_matrix.T)!=0)[0])
# 求 I|Iu
Iu = {}
for u in U:
    Iu.setdefault(u,[])
    Iu[u] = list(set(I)-set(np.where(train_matrix[u]!=0)[0]))

In [4]:
def predict(userId, itemI, itemJ):
    rui = np.dot(Uu[userId],Vi[itemI]) + bi[itemI]
    ruj = np.dot(Uu[userId],Vi[itemJ]) + bi[itemJ]
    return rui, ruj

In [5]:
def sigmoid(ruij):
    d = (1 + math.exp(-ruij))
    return 1.0 / d

In [6]:
def BPR(Rui, Uuk, Vik, bik, T, alpha_u, alpha_v, beta_v, gama):
    Rk = list()
    for u in U:
        ri = list(np.where(Rui[u,:]!=0)[0])
        for j in ri:
            Rk.append([u,j])
    Rk = np.array(Rk)
    for _ in tqdm(range(T)):
        K = np.random.permutation(Rk)
        for k in K:
            ui, uj = k
            # randomly pick up an item j from J|Ju
            idx = np.random.randint(len(Iu[ui]))
            ij = Iu[ui][idx]
            # Calculate relevant parameters
            rui, ruj = predict(ui, uj, ij)
            sgm = sigmoid(-(rui - ruj))
            # Calculate the gradient of each parameters
            grad_Uu = -sgm*(Vik[uj,:]-Vik[ij,:])+alpha_u*Uuk[ui,:]
            grad_Vi = -sgm*Uuk[ui,:]+alpha_v*Vik[uj,:]
            grad_Vj = -sgm*Uuk[ui,:]*(-1)+alpha_v*Vik[ij,:]
            grad_bi = -sgm + beta_v*bik[uj]
            grad_bj = -sgm*(-1) + beta_v*bik[ij] 
            # update the model parameters
            Uuk[ui,:] -= gama*grad_Uu
            Vik[uj,:] -= gama*grad_Vi
            Vik[ij,:] -= gama*grad_Vj
            bik[uj] -= gama*grad_bi
            bik[ij] -= gama*grad_bj
    return Uuk, Vik, bik

In [7]:
# evaluate function
def PreK(rank, truth, k):
    prek = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        preuk = 0
        ure+=1
        for j in range(k):
            if rank[i][j] in ls:
                preuk+=1
        prek+=preuk/k
    return prek/ure
def RecK(rank, truth, k):
    reck = 0
    ure = 0
    for i in range(n):
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        recuk = 0
        ure += 1
        for j in range(k):
            if rank[i][j] in ls:
                recuk += 1
        reck+=recuk/ts
    return reck/ure

In [8]:
gamma = 0.01
alpha_u = 0.01
alpha_v = 0.01
beta_v = 0.01
lr = 0.01
T = 500
d = 20
# bi、Uu、Vi
avg = np.sum(y_ui)/n/m
bi = np.sum(y_ui, axis=0)/n - avg
bi = np.insert(bi, 0, 0)
Uu = (np.random.rand(n+1, d) - 0.5) * 0.01
Vi = (np.random.rand(m+1, d) - 0.5) * 0.01
R = train_matrix.copy()
Uu, Vi, bi = BPR(R, Uu, Vi, bi, T, alpha_u, alpha_v, beta_v, gamma)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [13:03<00:00,  1.57s/it]


In [10]:
rank_bpr = list()
Ruj = np.dot(Uu,Vi.T)+bi
for i in tqdm(range(n)):
    exclude = list(np.where(train_matrix[i,:]!=0)[0])
    ls = list()
    for j in range(m):
        if j in exclude:
            continue
        ls.append((j,Ruj[i,j]))
    ls = sorted(ls, key=lambda x:x[1], reverse=True)
    rank_bpr.append([ls[j][0] for j in range(len(ls))])

100%|████████████████████████████████████████████████████████████████████████████████| 943/943 [00:12<00:00, 72.74it/s]


In [11]:
print("BPR Pre@5:"+str(PreK(rank_bpr, test_matrix, 5)))
print("BPR Rec@5:"+str(RecK(rank_bpr, test_matrix, 5)))

BPR Pre@5:0.376315789473684
BPR Rec@5:0.11535408765994516
